In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog
import random

# 读取数据
land_info = pd.read_excel("D:/pythonproject/data_science/excel_files/附件1.xlsx", sheet_name="乡村的现有耕地")
crop_info = pd.read_excel("D:/pythonproject/data_science/excel_files/附件1.xlsx", sheet_name="乡村种植的农作物")
planting_2023 = pd.read_excel("D:/pythonproject/data_science/excel_files/附件2.xlsx", sheet_name="2023年的农作物种植情况")
stats_2023 = pd.read_excel("D:/pythonproject/data_science/excel_files/附件2.xlsx", sheet_name="2023年统计的相关数据")

# 预处理数据
# 提取地块类型和面积
land_types = land_info[['地块名称', '地块类型', '地块面积/亩']].dropna()

# 提取作物信息
crop_types = crop_info[['作物编号', '作物名称', '作物类型', '种植耕地']].dropna()

# 提取2023年种植情况
planting_2023 = planting_2023[['地块名称', '作物编号', '作物名称', '作物类型', '种植面积/亩', '种植季次']].dropna()

# 提取2023年统计数据
stats_2023 = stats_2023[['作物编号', '作物名称', '地块类型', '种植季次', '亩产量/斤', '种植成本/(元/亩)', '销售单价/(元/斤)']].dropna()

# 定义地块类型和作物类型的映射关系
land_type_mapping = {
    '平旱地': ['粮食'],
    '梯田': ['粮食'],
    '山坡地': ['粮食'],
    '水浇地': ['水稻', '蔬菜'],
    '普通大棚': ['蔬菜', '食用菌'],
    '智慧大棚': ['蔬菜']
}

# 定义作物轮作规则
# 每种作物不能连续重茬种植
# 每个地块三年内至少种植一次豆类作物

# 定义经济效益计算函数
def calculate_profit(crop_id, land_type, season, area):
    # 获取亩产量、成本和单价
    stats = stats_2023[(stats_2023['作物编号'] == crop_id) & 
                      (stats_2023['地块类型'] == land_type) & 
                      (stats_2023['种植季次'] == season)]
    if stats.empty:
        return 0
    yield_per_mu = stats['亩产量/斤'].values[0]
    cost_per_mu = stats['种植成本/(元/亩)'].values[0]
    price_range = stats['销售单价/(元/斤)'].values[0]
    # 处理价格范围
    if '-' in price_range:
        min_price, max_price = map(float, price_range.split('-'))
        price = (min_price + max_price) / 2
    else:
        price = float(price_range)
    # 计算利润
    revenue = yield_per_mu * price * area
    cost = cost_per_mu * area
    profit = revenue - cost
    return profit

# 定义随机变量模拟函数
def simulate_random_variables(year):
    # 模拟小麦和玉米的销售量增长
    if year > 2023:
        growth_rate = random.uniform(0.05, 0.10)
    else:
        growth_rate = 0
    # 模拟其他作物的销售量变化
    other_crop_change = random.uniform(-0.05, 0.05)
    # 模拟亩产量变化
    yield_change = random.uniform(-0.10, 0.10)
    # 模拟种植成本增长
    cost_growth = 0.05
    # 模拟蔬菜价格增长
    vegetable_price_growth = 0.05
    # 模拟食用菌价格下降
    mushroom_price_decline = random.uniform(-0.05, -0.01)
    
    return {
        'growth_rate': growth_rate,
        'other_crop_change': other_crop_change,
        'yield_change': yield_change,
        'cost_growth': cost_growth,
        'vegetable_price_growth': vegetable_price_growth,
        'mushroom_price_decline': mushroom_price_decline
    }

# 定义蒙特卡洛模拟函数
def monte_carlo_simulation(years, num_simulations=1000):
    results = []
    for _ in range(num_simulations):
        simulation_result = {}
        for year in years:
            random_vars = simulate_random_variables(year)
            # 根据随机变量调整数据
            # 这里简化模型，实际需根据数据调整
            simulation_result[year] = random_vars
        results.append(simulation_result)
    return results

# 定义随机规划模型
def stochastic_programming(years, num_simulations=1000):
    # 生成模拟数据
    simulations = monte_carlo_simulation(years, num_simulations)
    # 求解每种情景的最优种植方案
    optimal_plans = []
    for sim in simulations:
        # 这里简化模型，实际需根据数据调整
        optimal_plans.append({
            'year': years,
            'plan': '示例种植方案'
        })
    return optimal_plans

# 执行随机规划模型
years = list(range(2024, 2031))
optimal_plans = stochastic_programming(years)

# 保存结果
result_df = pd.DataFrame(optimal_plans)
result_df.to_excel("result2.xlsx", index=False)